In [17]:
%load_ext autoreload
from transformers import TrainingArguments, Trainer, LlamaTokenizerFast, LlamaModel, LlamaForCausalLM, LlamaConfig
from transformers import DataCollatorForLanguageModeling
import transformers
import numpy as np
import evaluate
import datasets
from replacer import replace_linears_in_hf
from bitnet1 import BitLinear1B
from bitnet158 import BitLinear158B

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

dataset = datasets.load_dataset("wikitext", "wikitext-2-v1")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True,)

train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=4)
eval_dataset = eval_dataset.map(tokenize_function, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [24]:
configuration = LlamaConfig(
    intermediate_size=1024,
    hidden_size=1024,
    num_hidden_layers=4,
    num_attention_heads=4,
    max_position_embeddings=1024,
)

model = LlamaForCausalLM(configuration)
# model = LlamaModel(configuration)

# model = replace_linears_in_hf(model, BitLinear=BitLinear158B)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred: transformers.EvalPrediction):
    print("eval_pred", eval_pred)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions)

In [25]:

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

/home/candr/Documents/School/FinalNLP/.venv/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 122.38 MiB is free. Including non-PyTorch memory, this process has 5.36 GiB memory in use. Of the allocated memory 5.21 GiB is allocated by PyTorch, and 21.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
trainer.train()

  0%|          | 0/13770 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 25.88 MiB is free. Including non-PyTorch memory, this process has 5.45 GiB memory in use. Of the allocated memory 5.30 GiB is allocated by PyTorch, and 21.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)